In [ ]:
import glob
import os
from PIL import Image, ImageOps
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from keras.layers import Dropout, Input, Dense, Activation,GlobalMaxPooling2D, BatchNormalization, Flatten, Conv2D, MaxPooling2D
from keras.models import Model, load_model
from keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.callbacks import LearningRateScheduler, EarlyStopping
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

In [ ]:
z = glob.glob('../input/train/*/*.png')
ori_label = []
ori_imgs = []
for fn in z:
    if fn[-3:] != 'png':
        continue
    ori_label.append(fn.split('/')[-2])
    new_img = Image.open(fn)
    ori_imgs.append(ImageOps.fit(new_img, (64, 64), Image.ANTIALIAS).convert('RGB'))
    
    

In [ ]:
ori_imgs[2]

In [ ]:
imgs = np.array([np.array(im) for im in ori_imgs])
imgs = imgs.reshape(imgs.shape[0], 64,64, 3) / 255
lb = LabelBinarizer().fit(ori_label)
label = lb.transform(ori_label) 

In [ ]:
trainX, validX, trainY, validY = train_test_split(imgs, label, test_size=0.05, random_state=42)

In [ ]:
trainX.shape , validX.shape ,trainY.shape , validY.shape

In [ ]:
image_input=Input(shape=(64,64,3))
# create the base pre-trained model
#base_model = ResNet50(input_tensor=image_input,weights='imagenet', include_top=False)

base_model = MobileNet(input_tensor=image_input,weights='imagenet', include_top=False)

x = base_model.output
x = Flatten()(x)
predictions = Dense(12, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)


# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy',metrics=['accuracy'])

# train the model on the new data for a few epochs
batch_size = 32
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.8 ** x)
earlystop = EarlyStopping(patience=5)
modelsave = ModelCheckpoint(
    filepath='model.h5', save_best_only=True, verbose=1)
history = model.fit(
    trainX, trainY, batch_size=batch_size,
    epochs=200, 
    validation_data=(validX, validY),
    callbacks=[annealer, earlystop, modelsave]
)


In [ ]:
def printHistory(history,):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    f.subplots_adjust(top=0.85, wspace=0.3)

    ax1.plot( history.history['accuracy'], label='Train Accuracy')
    ax1.plot( history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_ylabel('Accuracy Value')
    ax1.set_xlabel('Epoch')
    ax1.set_title('Accuracy')
    l1 = ax1.legend(loc="best")

    ax2.plot( history.history['loss'], label='Train Loss')
    ax2.plot( history.history['val_loss'], label='Validation Loss')
    ax2.set_ylabel('Loss Value')
    ax2.set_xlabel('Epoch')
    ax2.set_title('Loss')
    l2 = ax2.legend(loc="best")
    
printHistory(history)

In [ ]:
z = glob.glob('../input/test/*.png')
test_imgs = []
names = []
for fn in z:
    if fn[-3:] != 'png':
        continue
    names.append(fn.split('/')[-1])
    new_img = Image.open(fn)
    test_img = ImageOps.fit(new_img, (64, 64), Image.ANTIALIAS).convert('RGB')
    test_imgs.append(test_img)
model = load_model('model.h5')

In [ ]:
timgs = np.array([np.array(im) for im in test_imgs])
testX = timgs.reshape(timgs.shape[0], 64, 64, 3) / 255

In [ ]:
yhat = model.predict(testX)
test_y = lb.inverse_transform(yhat)

In [ ]:
df = pd.DataFrame(data={'file': names, 'species': test_y})
df_sort = df.sort_values(by=['file'])
df_sort.to_csv('results.csv', index=False)